In [1]:
# %% Imports
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from numpy import clip, column_stack, argmax
import pandas as pd
import numpy as np

# %% Load dataset and create train-test sets
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# Extract features and target
X_train = train_data.iloc[:, :-1].to_numpy()  # Features
y_train = train_data.iloc[:, -1].to_numpy()   # Target
X_test = test_data.iloc[:, :-1].to_numpy()    # Test features
y_test = test_data.iloc[:, -1].to_numpy()     # Test target

In [2]:
# Define variable names
var_names = ["Feature_" + str(i) for i in range(X_train.shape[1])]

# Define the hyperparameter grid
param_grid = {
    'nr_clus': [3, 4, 5, 6, 7],  # Number of clusters
    'clustering_method': ['fcm', 'gk']  # Clustering methods
}

# Perform grid search
best_params = None
best_score = 0
best_metrics = {}

for nr_clus in param_grid['nr_clus']:
    for method in param_grid['clustering_method']:
        print(f"\nTesting parameters: nr_clus={nr_clus}, method={method}")

        # %% Cluster the input-output space
        cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=nr_clus)
        clust_centers, part_matrix, _ = cl.cluster(method=method)

        # Debugging: Check cluster centers and partition matrix
        print(f"Cluster Centers (Shape: {clust_centers.shape}):\n{clust_centers}")
        print(f"Partition Matrix (Shape: {part_matrix.shape}): First Row:\n{part_matrix[0]}")

        # %% Estimate membership functions parameters
        ae = AntecedentEstimator(X_train, part_matrix)
        antecedent_params = ae.determineMF()

        # Debugging: Print a summary of the membership functions
        print(f"Antecedent Parameters: {antecedent_params}")

        # %% Estimate consequent parameters
        ce = ConsequentEstimator(X_train, y_train, part_matrix)
        conseq_params = ce.suglms()

        # %% Build first-order Takagi-Sugeno model
        modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, var_names, save_simpful_code=False)
        model = modbuilder.get_model()

        # %% Get model predictions
        modtester = SugenoFISTester(model, X_test, var_names)
        y_pred_probs = clip(modtester.predict()[0], 0, 1)
        y_pred_probs = column_stack((1 - y_pred_probs, y_pred_probs))
        y_pred = argmax(y_pred_probs, axis=1)

        # Debugging: Check predictions
        print(f"Predictions (Sample): {y_pred[:10]}")

        # %% Compute classification metrics
        acc_score = accuracy_score(y_test, y_pred)
        rec_score = recall_score(y_test, y_pred)
        prec_score = precision_score(y_test, y_pred)
        F1_score = f1_score(y_test, y_pred)
        kappa = cohen_kappa_score(y_test, y_pred)

        print(f"Results: Accuracy={acc_score:.3f}, Precision={prec_score:.3f}, Recall={rec_score:.3f}, "
              f"F1={F1_score:.3f}, Kappa={kappa:.3f}")

        # Update best parameters if this configuration is better
        if acc_score > best_score:
            best_score = acc_score
            best_params = {'nr_clus': nr_clus, 'method': method}
            best_metrics = {
                'accuracy': acc_score,
                'precision': prec_score,
                'recall': rec_score,
                'f1': F1_score,
                'kappa': kappa
            }

# Output the best parameters and their metrics
print("\nBest Hyperparameters and Metrics:")
print(f"Best Parameters: {best_params}")
print(f"Metrics: {best_metrics}")


Testing parameters: nr_clus=3, method=fcm
Cluster Centers (Shape: (3, 96)):
[[4.62786043e-01 5.09861040e-01 5.08749878e-01 6.03204758e-01
  6.03214133e-01 9.98756833e-01 7.96995046e-01 8.08904232e-01
  3.03279175e-01 7.81194438e-01 1.87892249e+09 1.70954823e+09
  4.63878268e-01 1.00631362e+07 7.60930900e-02 1.75894919e-01
  1.75824731e-01 1.75918228e-01 2.08958057e-01 3.19931487e-01
  5.72519081e+05 9.84640828e-02 1.66345387e-01 2.22673629e-02
  8.47481556e-01 6.87673277e-01 6.87700566e-01 2.17427365e-01
  5.18522379e+09 5.60142814e+06 2.63741708e-01 3.76594208e-01
  1.12651254e-02 1.37392946e+07 6.30761652e-01 5.81370186e+06
  1.48798188e-01 8.51201812e-01 8.82938652e-03 3.82749719e-01
  8.06087293e-03 9.84837834e-02 1.65513612e-01 4.05833139e-01
  1.23561216e-01 8.05060301e+06 5.50662583e+06 2.02457286e+09
  1.40690112e+09 4.04576933e-02 8.38815629e+06 3.92243989e-01
  9.14824439e+06 7.84683584e-01 3.56461263e-01 5.01961768e-01
  8.70577734e-02 2.32015540e+06 1.13706956e+08 1.174423